In [ ]:
from image_util import resize_and_save

# Correct paths to your dataset
csv_file = r"./archive/data.csv"
base_folder = r"./archive/dataset"
target_folder = r"./archive/resized"

# resize images
resize_and_save(csv_file, base_folder, target_folder)


In [ ]:
from image_util import load_and_normalize, to_grayscale, to_histgram, to_pca, to_norm
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from logistic_regression import cross_validate_model, LogisticRegression

# Correct paths to your dataset
csv_file = r"./archive/data.csv"
base_folder = r"./archive/resized"

start_time = datetime.datetime.now()
print(f"Start Time {start_time}")

# load and normalize images
images, labels = load_and_normalize(csv_file, base_folder)
normalize_time = datetime.datetime.now()
print(f"loading time: {normalize_time - start_time}")

# Step 1 - GRAYSCALE
grayscale_images = to_grayscale(images)
grayscale_time = datetime.datetime.now()
print(f"grayscale time: {grayscale_time - normalize_time}")

# Step 2 - HISTGRAM
histgram_images = to_histgram(grayscale_images)
histgram_time = datetime.datetime.now()
print(f"histogram time: {histgram_time - grayscale_time}")

# Step 3 - PCA
pca_images = to_pca(histgram_images)
pca_time = datetime.datetime.now()
print(f"pca time: {pca_time - histgram_time}")

# Step 4 - Normalization
norm_images = to_norm(pca_images)
norm_time = datetime.datetime.now()
print(f"norm time: {norm_time - pca_time}")

# Step 5 - Encode labels to numerical values
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
# One-hot encode the labels for softmax regression
labels_one_hot = np.eye(len(np.unique(labels_encoded)))[labels_encoded]
one_hot_time = datetime.datetime.now()
print(f"label Encoding time: {one_hot_time - one_hot_time}")

# Step 6 - # Split into training, validation, and test sets
images_train_full, images_test, labels_train_full, labels_test = train_test_split(norm_images, labels_one_hot, test_size=0.2, random_state=42)
images_train, images_val, labels_train, labels_val = train_test_split(images_train_full, labels_train_full, test_size=0.2, random_state=42)
split_time = datetime.datetime.now()
print(f"split dataset time: {split_time - one_hot_time}")

# Step 7 - Cross validate the model
cross_validation_accuracy = cross_validate_model(norm_images, labels_one_hot, num_folds=5)
cross_validation_time = datetime.datetime.now()
print(f"cross validation time: {cross_validation_time - split_time}")

# Step 8 - Train the model
final_model = LogisticRegression(input_size=images_train_full.shape[1], num_classes=labels_train_full.shape[1], learning_rate=0.1,
                                 regularization=0.001)

final_model.train(images_train_full, labels_train_full, epochs=1000)
train_time = datetime.datetime.now()
print(f"train time: {train_time - cross_validation_time}")

# Step 9 Test the final model
labels_test_pred = final_model.predict(images_test)
final_accuracy = np.mean(np.argmax(labels_test, axis=1) == labels_test_pred)
print(f"Final Test Accuracy: {final_accuracy}")
testing_time = datetime.datetime.now()
print(f"testing time: {testing_time - train_time}")

end_time = datetime.datetime.now()
print(f"End Time: {end_time}")
print(f"Total Duration: {end_time - start_time}")
